In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer

## Auxiliary Functions

In [2]:
model_name_or_path = 'google/vit-large-patch16-224-in21k'
feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path)
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\binha\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\binha\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\binha\AppData\Local\Temp\ipykernel_5104\4174238103.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major ver

## Create Image Dataset

1. Put all images under the folder '/imgs'
2. Put training images in '/imgs/train/'
3. Put meta information of training images in '/imgs/train/metadata.csv'. The first column should be the file name corresponding to each image. The following columns could be any additional information, such as label.
4. Put validation images in '/imgs/val/'
5. Put meta information of validation images in '/imgs/val/metadata.csv'. The first column should be the file name corresponding to each image. The following columns could be any additional information, such as label.

In [3]:
dataset = load_dataset("imagefolder", data_dir="planet-imgs-original/split1/")
prepared_ds = dataset.with_transform(transform)
prepared_ds

Resolving data files:   0%|          | 0/125 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Using custom data configuration default-d65ce7d20941fc0b
Found cached dataset imagefolder (C:/Users/binha/.cache/huggingface/datasets/imagefolder/default-d65ce7d20941fc0b/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 124
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 39
    })
    validation: Dataset({
        features: ['image', 'labels'],
        num_rows: 31
    })
})

## Load Model

In [4]:
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=2
)

Some weights of the model checkpoint at google/vit-large-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir="./bufflegrass-finetune",
    per_device_train_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    num_train_epochs=10,
    fp16=True,
    learning_rate=1e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
)

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [7]:
train_results = trainer.train()

C:\Users\binha\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: binhan96816 (beanham). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.708700,0.647681,0.677419
2,0.579300,0.723397,0.645161
3,0.432900,0.880543,0.677419
4,0.463500,1.223809,0.709677
5,0.149100,1.706070,0.612903
6,0.188900,1.479576,0.612903
7,0.106200,1.542576,0.709677
8,0.115400,1.551901,0.677419
9,0.103100,1.645388,0.645161
10,0.102500,1.678675,0.612903


In [8]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.6774
  eval_loss               =     0.6477
  eval_runtime            = 0:00:07.52
  eval_samples_per_second =       4.12
  eval_steps_per_second   =      0.532


In [14]:
1-np.mean(dataset['test']['labels'])

0.5128205128205128

In [9]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =     0.5385
  eval_loss               =     0.6519
  eval_runtime            = 0:00:09.27
  eval_samples_per_second =      4.205
  eval_steps_per_second   =      0.539
